In [ ]:
import shapefile
import numpy as np
import pandas as pd
from scipy import stats
import plotly.graph_objects as go
import plotly.express as px

In [ ]:

sf = shapefile.Reader("/home/asif/Desktop/AQ/bgd_phy_forestnaturalparks_lged.zip")
print(sf.shapeType==shapefile.POLYGON)

shapes = sf.shapes()
print(len(shapes))
print(shapes[:5])

s = sf.shape(0)
print(s)

print(['%.3f' % coord for coord in s.bbox])

print(sf.fields)

for s in shapes[:5]:
    print(s.bbox[::-1])

bbox_array = np.array(s.bbox)
lon,lat = bbox_array[[0,2]],bbox_array[[1,3]]
print(lon,lat)

# for s1 in shapes:
#     for s2 in shapes:
#         if s1.bbox[0:2]==s2.bbox[3:4]:
#             print(s1,s2)

In [ ]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()

for s in shapes[:5]:
    bbox_array = np.array(s.bbox)
    lon,lat = bbox_array[[0,2]],bbox_array[[1,3]]

    fig.add_trace(go.Scattermapbox(
        mode = "markers+lines",
        lon = lon,
        lat = lat,
        marker = {'size': 10}))

fig.update_layout(
    height=900,
    width=900,
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': 91.84, 'lat': 24.13},
        'style': "open-street-map",
        # 'center': {'lon': -20, 'lat': -20},
        'zoom': 8})

fig.show()

In [ ]:
bbox_array_all = np.array([s.bbox[::-1] for s in shapes])
forest_location = pd.DataFrame(((bbox_array_all[:,:2]+bbox_array_all[:,2:4])/2),columns=["Latitude","Longitude"])
forest_location

In [ ]:
forest_location.to_csv("forest_location.csv",index=False)

In [ ]:
near_forest_location = ["Thakurgaon_Rangpur", "Dinajpur_Rangpur", "Saidpur_Rajshahi", "Rangpur_Rangpur",
                        "Lalmanirhat_Rangpur", "Sherpur_Mymensingh", "Jamalpur_Mymensingh", "Sarishabari_Mymensingh",
                        "Mymensingh_Mymensingh", "Tangail_Dhaka", "Nagarpur_Dhaka", "Habiganj_Sylhet",
                        "Satkhira_Khulna", "Khagrachhari_Chittagong", "Chittagong_Chittagong", "Patiya_Chittagong",
                        "Satkania_Chittagong", "Sandwip_Chittagong", "Joypur Hat_Rajshahi", "Netrakona_Mymensingh",
                        "Comilla_Chittagong", "Maulavi Bazar_Sylhet", "Bhola_Barisal", "Feni_Chittagong"
                        ]

In [ ]:
from data_preparation.spatio_temporal_filtering import read_bd_data_4_years

metadata, series, metadata_region, region_series, metadata_country, country_series = read_bd_data_4_years()
series

In [ ]:
series[near_forest_location].stack().mean()

In [ ]:
series[series.columns[~series.columns.isin(near_forest_location)]].stack().mean()

In [ ]:
duplicates_bd = pd.read_csv("../data_preparation/Extra Data/duplicates_bd.csv",index_col="zone")
duplicates_bd

In [ ]:
intro_table = metadata[["Zone","Region"]].copy()
intro_table["Average Reading"] = series.mean().round(2)
intro_table["Missing Entries"] = series.isna().sum()
intro_table["Duplicate Values"] = duplicates_bd.values
intro_table = intro_table.set_index("Zone").sort_values(by="Average Reading",ascending=False)
# intro_table["Near Forest"] = "No"
# intro_table.loc[near_forest_location,"Near Forest"] = "Yes"
print(intro_table.to_latex(col_space=3))

In [ ]:
intro_table

In [ ]:
zone_income = pd.read_csv("../data_preparation/Extra Data/zone_income.csv",index_col="zone")
zone_income_index_full = zone_income.index.copy()
zone_income.index = zone_income.index.str.split("_").str[0]
zone_income

In [ ]:
intro_table_zone_income = pd.concat((intro_table,zone_income),axis=1)
intro_table_zone_income

In [ ]:
intro_table_zone_income = intro_table_zone_income.drop_duplicates(subset=["per_capita_income"])
intro_table_zone_income

In [ ]:
intro_table_zone_income["Average Reading"].corr(intro_table_zone_income["per_capita_income"])

In [ ]:
intro_table_zone_income.isna().sum()

In [ ]:


def create_scatter_regression_plot(df):
    # Perform linear regression
    slope, intercept, r_value, p_value, std_err = stats.linregress(
        df['per_capita_income'],
        df['Average Reading']
    )

    # Create regression line
    line_x = np.array([df['per_capita_income'].min(), df['per_capita_income'].max()])
    line_y = intercept + slope * line_x

    # Create scatter plot with regression line
    fig = go.Figure()

    # Scatter plot
    fig.add_trace(go.Scatter(
        x=df['per_capita_income'],
        y=df['Average Reading'],
        mode='markers',
        name='Data Points',
        marker=dict(
            size=12,
            color=df['per_capita_income'],
            colorscale='gray',
            # showscale=True,
            # colorbar=dict(title='Per Capita Income')
        ),
        hovertemplate=(
            'Per Capita Income: %{x}<br>' +
            'Average Reading: %{y}<extra></extra>'
        )
    ))

    # Regression line
    fig.add_trace(go.Scatter(
        x=line_x,
        y=line_y,
        mode='lines',
        name='Regression Line',
        line=dict(color='royalblue', dash='dash')
    ))

    fig.update_traces(marker=dict(line=dict(width=1, color='black')))
    # Update layout
    fig.update_layout(
        title=f'Scatter Plot: Average Reading vs Per Capita Income<br>' +
              f'R-squared: {r_value**2:.4f}, p-value: {p_value:.4f}',
        xaxis_title='Per Capita Income',
        yaxis_title='Average Reading',
        template='plotly_white',
        width = 900,
        height = 900
    )

    return fig

# Example usage
plot = create_scatter_regression_plot(intro_table_zone_income)
plot.show()

In [ ]:
import plotly.express as px

# Example data
df = intro_table_zone_income[['Average Reading', 'per_capita_income']]

# Create a 2D KDE plot
fig = px.density_contour(
    df,
    y="Average Reading",
    x="per_capita_income",
    title="2D KDE of Average Reading vs Per Capita Income",
    labels={"Average Reading": "Average Reading", "per_capita_income": "Per Capita Income"},
    template="plotly_white",
)

# Add filled contours for better visualization
fig.update_traces(contours_coloring="fill", colorscale="Greys")

# Customize layout
fig.update_layout(
    yaxis_title="Average Reading",
    xaxis_title="Per Capita Income",
    legend_title="Density",
    height=900,
    width=900
)

# Show the plot
fig.show()

# Zone to District Mapping

In [ ]:
district_income = pd.read_csv("../data_preparation/Extra Data/district_income.csv",index_col="district", sep='\t+', engine='python')[["per_capita_income"]]
district_income

In [ ]:
zone_income_full = zone_income.set_index(zone_income_index_full).reset_index().rename(columns={"index": "zone"})
district_income = district_income.reset_index().rename(columns={"index": "district"})

In [ ]:
zone_district_map = pd.merge(zone_income_full, district_income, on="per_capita_income", how="inner")[["zone","district"]]
# zone_district_map.to_csv("zone_district_map.csv",index=False)
zone_district_map

# BBS Data Analysis

In [ ]:
Household_Population_Literacy_Rate_District_2011 = pd.read_csv('../data_preparation/Extra Data/BBS 2022 Data/Household_Population_Literacy_Rate_District_2011.csv', sep='\s+', header=[0], index_col=0).replace("-",0).astype(float)
Household_Population_Literacy_Rate_District_2011

In [ ]:
Household_Population_Literacy_Rate_District_2011.columns

In [ ]:
road_length_district_2022 = pd.read_csv('../data_preparation/Extra Data/BBS 2022 Data/road_length_district_2022.csv', sep='\s+', header=[0], index_col=0).replace("-",0).astype(float)
road_length_district_2022

In [ ]:
road_length_district_2022.columns

In [ ]:
district_income.index.sort_values()

In [ ]:
set(road_length_district_2022.index) - set(Household_Population_Literacy_Rate_District_2011.index)

In [ ]:
zone_district_map

In [ ]:
average_reading = series.median()[zone_district_map.zone]
average_reading.name = "average_reading"
average_reading

In [ ]:
road_and_household_data = pd.concat((road_length_district_2022.loc[zone_district_map.district],Household_Population_Literacy_Rate_District_2011.loc[zone_district_map.district]),axis=1)
road_and_household_data["Per Capita Income"] = zone_income["per_capita_income"].values
road_and_household_data.index = average_reading.index
road_and_household_data

In [ ]:
# household_data = road_and_household_data[Household_Population_Literacy_Rate_District_2011.columns]
road_and_household_data.apply(lambda x: x.corr(average_reading))

In [ ]:
Households_Total_Unique_Sorted = road_and_household_data["Population"].sort_values() # .drop_duplicates()
Households_Total_Unique_Sorted.describe() # 477976  714971

In [ ]:
fig = px.line(Households_Total_Unique_Sorted)
fig.show()

In [ ]:
urban_areas = road_and_household_data["Population"]>2.5e+06
rural_areas = ~urban_areas
rural_areas.sum(),urban_areas.sum()

In [ ]:
urban_rural_areas = urban_areas
urban_rural_group = urban_rural_areas.to_frame().groupby("Population")
urban_rural_average = []

for g,areas_of_type in urban_rural_group:
    # print(areas_of_type)
    name = "Urban" if g else "Rural"
    area_reading = series[areas_of_type.index].mean(axis=1).rename(name).to_frame()

    # Extract the month name or month number
    area_reading['month'] = area_reading.index.month
    # print(area_reading)

    # Group by month and calculate the average
    monthly_avg = area_reading.groupby('month')[name].mean()
    print(monthly_avg)
    urban_rural_average.append(monthly_avg)

In [117]:
urban_rural_average_df = pd.concat(urban_rural_average,axis=1)
urban_rural_average_df

,Rural,Urban
month,,
1,123.025866,128.077374
2,99.940860,103.842861
3,74.362502,77.546485
4,47.321421,49.701635
5,31.119407,32.574720
6,26.363871,27.272845
7,21.362619,22.024330
8,21.532997,22.242965
9,23.555102,24.350402


In [128]:
# Calculate percentage change
df_pct_change = urban_rural_average_df.pct_change()

# Calculate the percentage change for the first month (January) using December's values
first_row_pct_change = (urban_rural_average_df.iloc[0] - urban_rural_average_df.iloc[-1]) / urban_rural_average_df.iloc[-1]

# Replace the NaN in the first row with the calculated values
df_pct_change.iloc[0] = first_row_pct_change

# Multiply by 100 to get actual percentage values
df_pct_change = df_pct_change * 100

# Round to two decimal places
df_pct_change = df_pct_change.round(2)
df_pct_change

,Rural,Urban
month,,
1,18.21,19.33
2,-18.76,-18.92
3,-25.59,-25.32
4,-36.36,-35.91
5,-34.24,-34.46
6,-15.28,-16.28
7,-18.97,-19.24
8,0.80,0.99
9,9.39,9.47


In [129]:
df_pct_change["Rural"]-df_pct_change["Urban"]

month
1    -1.12
2     0.16
3    -0.27
4    -0.45
5     0.22
6     1.00
7     0.27
8    -0.19
9    -0.08
10    1.97
11   -0.78
12   -0.51
dtype: float64

In [ ]:
road_and_household_data.columns

In [ ]:
road_and_household_data_log_transformed = road_and_household_data.apply(lambda x: np.log(x + 1))
average_reading_log = np.log(average_reading + 1)

road_and_household_data_selected_columns = [
    'Total Highways', 'Per Capita Income', 'Households Total',
    'Households General', 'Households Institutional', 'Households Others',
    'Population', 'Male', 'Female']

In [ ]:
from plotly.subplots import make_subplots

rows, cols = 3, 3

# Create subplots
fig = make_subplots(rows=rows, cols=cols, subplot_titles=road_and_household_data_selected_columns,
                    horizontal_spacing=0.05,  # Reduce horizontal spacing
                    vertical_spacing=0.1)

# Loop through each column and add scatter plots
for i, col in enumerate(road_and_household_data_selected_columns):
    # print("i ",i,col)
    row = i // cols + 1
    col_num = i % cols + 1

    # Create scatter plot with regression line
    scatter_fig = px.scatter(
        # x=road_and_household_data[col],
        x=road_and_household_data_log_transformed[col],
        y=average_reading,
        trendline="ols",
        labels={col: col, "y": "Average Reading"},
        title=f"{col} vs Average Reading"
    )

    scatter_fig.update_traces(
        marker=dict(
            color='silver',  # White fill for the markers
            line=dict(
                color='black',  # Black border for the markers
                width=1  # Width of the border line
            ),
            size=9  # Increase marker size (you can adjust this value as needed)

        )
    )

    scatter_fig.update_traces(
        line=dict(
            color='darkgrey',  # Set the trendline color to dark blue (or any other color)
            width=3  # Increase the width of the trendline
        ),
        selector=dict(mode='lines')  # Ensure the trendline settings are applied
    )

    # Add traces to the subplots
    for trace in scatter_fig['data']:
        # print(row,col_num)
        fig.add_trace(trace, row=row, col=col_num)

# Update layout
fig.update_layout(
    title="Scatter Plots with Regression Lines",
    height=1200, width=1200,
    showlegend=False,
    template='plotly_white',
    font_size = 15
)

# Show plot
fig.show()

In [ ]:
import statsmodels.api as sm

# Assuming 'road_and_household_data_log_transformed' is your DataFrame and 'average_reading' is the target variable

# Loop through all columns of the dataframe
for col in road_and_household_data_selected_columns:
    # Set the independent variable X and dependent variable y
    X = road_and_household_data_log_transformed[col]
    y = average_reading

    # Add constant to X for the intercept
    X = sm.add_constant(X)

    # Fit the OLS model
    model = sm.OLS(y, X).fit()

    # Print the OLS summary for each variable
    print(f"OLS Summary for {col}:")
    print(model.summary())
    print("\n" + "-"*80 + "\n")